In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from func import *
from numpy import exp
from scipy import ndimage
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from shutil import copyfile

In [2]:
######################## Scale Factor Error Flux Maps #####################
def err_ratio(data,err):
    data[2:-2,2:-2]=0
    err[2:-2,2:-2]=0
    edge_fluxmap = data[data!=0]
    edge_errmap = err[err!=0]
    err_fluxmap = np.std(edge_fluxmap)
    err_errmap = np.mean(edge_errmap)
    fact = err_fluxmap/err_errmap
    return fact

def flux_dat(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj))
    Hb1 = hdu[5].data
    Hb2 = hdu[6].data
    OIII_br = hdu[3].data
    OIII_nr = hdu[2].data
    hdu.close()
    return Hb1,Hb2,OIII_br,OIII_nr

def flux_err(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    Hb1 = hdu[5].data
    Hb2 = hdu[6].data
    OIII_br = hdu[3].data
    OIII_nr = hdu[2].data
    hdu.close()
    return Hb1,Hb2,OIII_br,OIII_nr

In [3]:
def test_tab(obj,Hb1,Hb2,wing,core,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(Hb1,name='Hb1'))
    hdus.append(fits.ImageHDU(Hb2,name='Hb2'))
    hdus.append(fits.ImageHDU(wing,name='OIII_wing'))
    hdus.append(fits.ImageHDU(core,name='OIII_core'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/subcube_par_test_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def test_err_tab(obj,Hb1_err,Hb2_err,wing_err,core_err,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(Hb1_err,name='Hb1_err'))
    hdus.append(fits.ImageHDU(Hb2_err,name='Hb2_err'))
    hdus.append(fits.ImageHDU(wing_err,name='OIII_wing_err'))
    hdus.append(fits.ImageHDU(core_err,name='OIII_core_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/subcube_par_err_test_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def flux_data_err(obj,emp_Hb1,emp_Hb2,emp_wing,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/subcube_par_test_%s.fits'%(destination_path_cube,obj,obj))
    (OIII_nr,OIII_br,Hb1_blr_br,Hb2_blr_br) = (hdu[4].data,hdu[3].data,hdu[1].data,hdu[2].data)
    hdu.close()
    
    hdu = fits.open('%s/Flux Maps/%s/subcube_par_err_test_%s.fits'%(destination_path_cube,obj,obj))
    (OIII_nr_err,OIII_br_err,Hb1_blr_br_err,Hb2_blr_br_err) = (hdu[4].data,hdu[3].data,hdu[1].data,hdu[2].data)
    hdu.close()
       
    (amp_OIII_nr,amp_OIII_br,amp_Hb1_blr_br,amp_Hb2_blr_br) = (np.max(OIII_nr),np.max(OIII_br),np.max(Hb1_blr_br),np.max(Hb2_blr_br))
    if amp_Hb1_blr_br > amp_Hb2_blr_br:
        (Hb_blr_br,amp_Hb_blr_br,Hb_blr_err,emp_Hb_blr) = (Hb1_blr_br,amp_Hb1_blr_br,Hb1_blr_br_err,emp_Hb1)
    else:
        (Hb_blr_br,amp_Hb_blr_br,Hb_blr_err,emp_Hb_blr) = (Hb2_blr_br,amp_Hb2_blr_br,Hb2_blr_br_err,emp_Hb2)
    (blr_err_final,wing_err_final,core_err_final) = (emp_Hb_blr*Hb_blr_err,emp_Hb_blr*OIII_br_err,emp_Hb_blr*OIII_nr_err)
    (Hb_blr_br,OIII_br,OIII_nr) = (np.random.normal(Hb_blr_br,blr_err_final),np.random.normal(OIII_br,wing_err_final),np.random.normal(OIII_nr,core_err_final))

    return Hb_blr_br,OIII_br,OIII_nr,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr,blr_err_final,wing_err_final,core_err_final

def remove_bad_pixel(data,err):
    data[data == 0.01] = 0
    err[err < 10**(-14)] =10**(10)  
    return data,err

def ellip_moffat2D(p,xy,data,error):
    xy=(x,y)
    [amp, x0, y0, A, B, C, alpha] = p 
    g = amp*((1.0+A*(x-x0)**2+B*(y-y0)**2+C*(x-x0)*(y-y0))**(-alpha))
    h = (data - g)/error
    return h.ravel()

def ellip_moffat2D_fixkin(p,xy,data,error,fixed_param):
    [A,B,C,alpha] = fixed_param
    xy=(x,y)
    [amp, x0, y0] = p    
    g = amp*((1.0+(A*((x-x0)**2))+(B*((y-y0)**2))+(C*(x-x0)*(y-y0)))**(-alpha))
    h = (data - g)/error
    return h.ravel()

def elliptical_moffat_fit(data,error,box_size,amp,x0,y0,muse_sampling_size,fix_par,MC_loops=100):
    y, x = np.mgrid[:box_size, :box_size]
    xy=(x,y)
    
    if fix_par is None:
        p= [amp,x0,y0,0.1,0.1,-0.00008,3]
        popt,pcov = leastsq(ellip_moffat2D,x0=p,args=(xy,data,error),maxfev = 10000000)
        [amp_out,x0_out,y0_out,A_out,B_out,C_out,alpha_out]= popt
        theta_out = 0.5*np.arctan(C_out/(A_out - B_out))
        a_out = np.sqrt(2/(A_out + B_out + np.sqrt(C_out**2 +(A_out - B_out)**2)))
        b_out = np.sqrt(2/(A_out + B_out - np.sqrt(C_out**2 +(A_out - B_out)**2)))
        [fwhm1_out,fwhm2_out] = [2*a_out*np.sqrt(2**(1/alpha_out)-1),2*b_out*np.sqrt(2**(1/alpha_out)-1)]
        par = [amp_out,x0_out,y0_out,A_out,B_out,C_out,alpha_out,a_out,b_out,theta_out,fwhm1_out,fwhm2_out]
        mod_rav = data.ravel() - ellip_moffat2D(popt,xy,data,error)*error.ravel()
    else:
        p= [amp,x0,y0]
        popt,pcov = leastsq(ellip_moffat2D_fixkin,x0=p,args=(xy,data,error,fix_par),maxfev = 10000000)
        [amp_out,x0_out,y0_out]= popt
        par = popt
        mod_rav = data.ravel() - ellip_moffat2D_fixkin(popt,xy,data,error,fix_par)*error.ravel()
    model = mod_rav.reshape(box_size,box_size)
    residual = data-model
    res = (residual/error)

    parameters_MC = np.zeros((len(par),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data,error) 
        if fix_par is None:
            p= [amp,x0,y0,0.1,0.1,-0.00008,3]
            popt_MC,pcov_MC = leastsq(ellip_moffat2D,x0=p,args=(xy,iteration_data,error),maxfev = 10000000)
            [amp_MC,x0_MC,y0_MC,A_MC,B_MC,C_MC,alpha_MC]= popt_MC
            theta_MC = 0.5*np.arctan(C_MC/(A_MC - B_MC))
            a_MC = np.sqrt(2/(A_MC + B_MC + np.sqrt(C_MC**2 +(A_MC - B_MC)**2)))
            b_MC = np.sqrt(2/(A_MC + B_MC - np.sqrt(C_MC**2 +(A_MC - B_MC)**2)))
            [fwhm1_MC,fwhm2_MC] = [2*a_MC*np.sqrt(2**(1/alpha_MC)-1),2*b_MC*np.sqrt(2**(1/alpha_MC)-1)]
            par_MC = [amp_MC,x0_MC,y0_MC,A_MC,B_MC,C_MC,alpha_MC,a_MC,b_MC,theta_MC,fwhm1_MC,fwhm2_MC]   
            parameters_MC[:,l] = par_MC    
        else:
            p= [amp,x0,y0]
            popt_MC,pcov_MC = leastsq(ellip_moffat2D_fixkin,x0=p,args=(xy,iteration_data,error,fix_par),maxfev = 10000000)
            [amp_MC,x0_out,y0_out]= popt_MC
            parameters_MC[:,l] = popt_MC    
    parameters_err = np.std(parameters_MC,1) 
    return par,parameters_err,model,res

In [4]:
############################### Extract all the 3" parameters###########################
def red_data_err(data,model,res,x0,y0,x_m,y_m,muse_sampling_size,box_size=4):
    if muse_sampling_size == 0.4:
        (y_cen,x_cen)=(int(round(y_m)),int(round(x_m)))
        data_red = data[y_cen-box_size:y_cen+box_size+1,x_cen-box_size:x_cen+box_size+1]
        mod_red = model[y_cen-box_size:y_cen+box_size+1,x_cen-box_size:x_cen+box_size+1]
        residual_red = res[y_cen-box_size:y_cen+box_size+1,x_cen-box_size:x_cen+box_size+1]
        (x0,y0)=(x0-7,y0-7)
    else:
        (y_cen,x_cen)=(int(round(y_m)),int(round(x_m)))
        data_red = data[y_cen-2*box_size:y_cen+2*box_size+1,x_cen-2*box_size:x_cen+2*box_size+1]
        mod_red =model[y_cen-2*box_size:y_cen+2*box_size+1,x_cen-2*box_size:x_cen+2*box_size+1]
        residual_red =res[y_cen-2*box_size:y_cen+2*box_size+1,x_cen-2*box_size:x_cen+2*box_size+1]
        (x0,y0)=(x0-14,y0-14)
    #plt.imshow(data_red,origin='lower')
    #plt.show()
    err_red = (data_red - mod_red)/residual_red
    return data_red,mod_red,residual_red,err_red,x0,y0 

In [5]:
####################### Extract the chi squared #########################################
def res_filter(res):
    res[res>10**10]=0
    return res

def chi_squared(data,model,err):
    res = data - model
    a = np.sum((res/err)**2)
    return a

def red_chi_squared(data,model,err,n_free):
    dof = len(data.flatten()) - n_free
    res = data - model
    k = res/err
    k[k>10**10]=0
    l = k[k!=0]
    a = np.sum(l**2)
    red = a/dof
    return red

In [6]:
def central_table(obj,parm_err,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'red_chisq_blr':0,'red_chisq_wing':1,'red_chisq_core':2,'norm_chisq_wing':3,'norm_chisq_core':4,
                  'psf_amp_blr':5,'psf_amp_wing':6,'psf_amp_core':7,'flux_wing_data':8,'flux_wing_model':9,
                  'flux_ratio':10,'FWHM1':11,'FWHM2':12,'e':13,'Hb_x':14,'Hb_y':15,'OIII_br_x':16,'OIII_br_y':17,
                  'OIII_nr_x':18,'OIII_nr_y':19,'off_wing_x':20,'off_wing_y':21,'off_core_x':22,'off_core_y':23}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key+'_err',format='E',array=[parm_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_parameters_err_alternative.fits'%(destination_path_cube,obj,obj),overwrite=True)

def source_name_tab(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    a1 = np.array(['%s'%(obj)])
    hdu = fits.BinTableHDU.from_columns([fits.Column(name='Source', format='20A', array=a1)])
    hdu.writeto('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),overwrite=True)  
    
def fluxden_chisq_table(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_parameters_err_alternative.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_parameters_err_alternative.fits'%(destination_path_cube,obj,obj),overwrite=True)

In [7]:
def algorithm_script(obj,z,broad2,p_init,box_size=11,min_wave=4750,max_wave=5090,prefix_path_cube="/home/mainak/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))
    (wo_data,wo_err,wo_wave,wo_header) = loadCube('%s/%s/%s.wo_absorption.fits'%(destination_path_cube,obj,obj)) 
    (cont_cube,cont_err,cont_wave,cont_header) = loadCube('%s/MUSE/%s/fitting/full/%s.cont_model.fits'%(prefix_path_cube,obj,obj))
    MC_loops = 100
    Monte_Carlo_loops = 100
    popt_MC = np.zeros((24,Monte_Carlo_loops))
    muse_sampling_size = sampling_size(cont_cube)
    [x0,y0] = agn_location(obj)
    [brightest_pixel_x,brightest_pixel_y] = [int(x0),int(y0)]
        
    for l in range(Monte_Carlo_loops):
        (Hb1,Hb2,wing,core) = flux_dat(obj)
        (Hb1_err,Hb2_err,wing_err,core_err) = flux_err(obj)
        (Hb1,Hb2,wing,core) = (np.random.normal(Hb1,Hb1_err),np.random.normal(Hb2,Hb2_err),np.random.normal(wing,wing_err),np.random.normal(core,core_err))
        test_tab(obj,Hb1,Hb2,wing,core)
        test_err_tab(obj,Hb1_err,Hb2_err,wing_err,core_err)
        (emp_Hb1,emp_Hb2,emp_wing) = (err_ratio(Hb1,Hb1_err),err_ratio(Hb2,Hb2_err),err_ratio(wing,wing_err))
             
        (Hb_blr_br_dat,OIII_br_dat,OIII_nr_dat,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr,Hb_blr_br_error,OIII_br_error,OIII_nr_error) = flux_data_err(obj,emp_Hb1,emp_Hb2,emp_wing)
        box_length = np.shape(Hb_blr_br_dat)[1]
        #print (box_length)
        (Hb_blr_br_data,Hb_blr_br_err) = remove_bad_pixel(Hb_blr_br_dat,Hb_blr_br_error)
        (OIII_br_data,OIII_br_err) = remove_bad_pixel(OIII_br_dat,OIII_br_error)
        (OIII_nr_data,OIII_nr_err) = remove_bad_pixel(OIII_nr_dat,OIII_nr_error)  
        [brightest_pixel_Hb_blr_br_y,brightest_pixel_Hb_blr_br_x] = ndimage.measurements.maximum_position(Hb_blr_br_data)
        [brightest_pixel_OIII_br_y,brightest_pixel_OIII_br_x] = ndimage.measurements.maximum_position(OIII_br_data)
        [brightest_pixel_OIII_nr_y,brightest_pixel_OIII_nr_x] = ndimage.measurements.maximum_position(OIII_nr_data) 
        
        (Hb_par,Hb_error,Hb_model,Hb_res) = elliptical_moffat_fit(Hb_blr_br_data,Hb_blr_br_err,box_length,amp_Hb_blr_br,brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,muse_sampling_size,None,100)
        fixed_param = [Hb_par[3],Hb_par[4],Hb_par[5],Hb_par[6]]
        (OIII_br_par,OIII_br_error,OIII_br_model,OIII_br_res) = elliptical_moffat_fit(OIII_br_data,OIII_br_err,box_length,amp_OIII_br,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,muse_sampling_size,fixed_param,100)   
        (OIII_nr_par,OIII_nr_error,OIII_nr_model,OIII_nr_res) = elliptical_moffat_fit(OIII_nr_data,OIII_nr_err,box_length,amp_OIII_nr,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y,muse_sampling_size,fixed_param,100)   
        (FWHM1,FWHM2) = (Hb_par[10],Hb_par[11])
        e = np.sqrt(1-(FWHM1**2/FWHM2**2))
        (Hb_cen_x,Hb_cen_y) = (Hb_par[1],Hb_par[2])
        (OIII_br_cen_x,OIII_br_cen_y) = (OIII_br_par[1],OIII_br_par[2])
        (OIII_nr_cen_x,OIII_nr_cen_y) = (OIII_nr_par[1],OIII_nr_par[2])
        
        (Hb_amp,OIII_br_amp,OIII_nr_amp) = (Hb_par[0],OIII_br_par[0],OIII_nr_par[0])
        half_box_shape = int((Hb_model.shape[0] - 1)/2)
        (y_m,x_m) = (half_box_shape,half_box_shape)
            #print (x_m,y_m)
        (Hb_data,Hb_model,Hb_res,Hb_err,Hb_x,Hb_y) = red_data_err(Hb_blr_br_data,Hb_model,Hb_res,Hb_cen_x,Hb_cen_y,x_m,y_m,muse_sampling_size,box_size=4)
        (OIII_br_data,OIII_br_model,OIII_br_res,OIII_br_err,OIII_br_x,OIII_br_y) = red_data_err(OIII_br_data,OIII_br_model,OIII_br_res,OIII_br_cen_x,OIII_br_cen_y,x_m,y_m,muse_sampling_size,box_size=4)
        (OIII_nr_data,OIII_nr_model,OIII_nr_res,OIII_nr_err,OIII_nr_x,OIII_nr_y) = red_data_err(OIII_nr_data,OIII_nr_model,OIII_nr_res,OIII_nr_cen_x,OIII_nr_cen_y,x_m,y_m,muse_sampling_size,box_size=4)
        (y_m,x_m) = ndimage.measurements.maximum_position(Hb_data)
        #print (x_m,y_m)
        chi_squared_Hb = chi_squared(Hb_data,Hb_model,Hb_err)
        chi_squared_OIII_br = chi_squared(OIII_br_data,OIII_br_model,OIII_br_err)
        chi_squared_OIII_nr = chi_squared(OIII_nr_data,OIII_nr_model,OIII_nr_err)

        red_chi_squared_Hb = red_chi_squared(Hb_data,Hb_model,Hb_err,7)
        red_chi_squared_OIII_br = red_chi_squared(OIII_br_data,OIII_br_model,OIII_br_err,3)
        red_chi_squared_OIII_nr = red_chi_squared(OIII_nr_data,OIII_nr_model,OIII_nr_err,3)

        normalized_chi_squared_OIII_br = (red_chi_squared_OIII_br/red_chi_squared_Hb)
        normalized_chi_squared_OIII_nr = (red_chi_squared_OIII_nr/red_chi_squared_Hb)
    
        flux_wing_data = np.sum(OIII_br_data)
        flux_wing_model = np.sum(OIII_br_model)
        flux_ratio = (flux_wing_data/flux_wing_model)
        
        (offset_wing_x,offset_wing_y) = (OIII_br_x - Hb_x,OIII_br_y - Hb_y)
        (offset_core_x,offset_core_y) = (OIII_nr_x - Hb_x,OIII_nr_y - Hb_y)
        popt = [red_chi_squared_Hb,red_chi_squared_OIII_br,red_chi_squared_OIII_nr,normalized_chi_squared_OIII_br,normalized_chi_squared_OIII_nr,Hb_amp,OIII_br_amp,OIII_nr_amp,flux_wing_data,flux_wing_model,flux_ratio,FWHM1,FWHM2,e,Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y,offset_wing_x,offset_wing_y,offset_core_x,offset_core_y]
        #print (popt)
        popt_MC[:,l] = popt
    parm_err = np.std(popt_MC,1)
    print ('norm chi-sq err and flux_err is', parm_err)
    #plt.imshow((OIII_br_data - OIII_br_model)/OIII_br_err,origin='lower')
    #plt.show()
    central_table(obj,parm_err)
    source_name_tab(obj)
    fluxden_chisq_table(obj)



In [8]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}

p_init= {'HE0021-1810':[0.1,1.5248,16056,50.0,1.0,3.0,15956,100.0,1.0,1.0,16056,1000.0,0,0,16056,1000.0,-0.001,2.0]
        ,'HE0021-1819':[1,12.5,15959,50.0,3,3,15759,200.0,2,2,15959,1000.0,0,0,15959,1000.0,-0.001,0.1]
        ,'HE0040-1105':[1.139,1.5,12507.0,50.0,1.0,3.0,12307.0,200.0,1.0,1.0,12507,1000.0,0,0,12507,1000.0,-0.001,2.0]
        ,'HE0108-4743':[1.7,4.5,7160.0,40.0,0.12,2.5,7100.0,120.0,1.0,1.0,6960,1300.0,1.0,1.0,7190,430.0,-0.001,2.0]
        ,'HE0114-0015':[0.1,1.5,13680,50.0,1.0,3.0,13480,100.0,1.0,1.0,13680,1000.0,0,0,13680,1000.0,-0.001,0.3]
        ,'HE0119-0118':[1,12.5,16302,50.0,3,3,16102,100.0,2,2,16302,1000.0,1,1,16302,2500.0,-0.001,0.1]
        ,'HE0212-0059':[0.1,1.0,7930,170.0,0.1,0.1,7630,450.0,1.0,1.0,7620,4000.0,0.1,0.1,4650,1090.0,-0.001,2.0]
        ,'HE0224-2834':[0.2,2.5,17940,50.0,0.3,0.3,17640,200.0,0.1,0.1,17940,1500.0,0,0,17940,1500.0,-0.001,0.1]
        ,'HE0227-0913':[3,6,4785,150,2,7,4385,300.0,7,1.8,4935,400.0,3,0.5,4735,1500.0,-0.3,4.7]
        ,'HE0232-0900':[2,40,12942,50.0,5,5,12742,100.0,5,1,12942,1000.0,0,0,12942,1000.0,-0.001,0.1]                                     
        ,'HE0253-1641':[2,10,9476,110.0,1,6,9176,370.0,1,1,9476,1500.0,0.5,0.5,9476,550.0,-0.001,0.5]
        ,'HE0345+0056':[0.5,20,9300,150,5,10,9000,300,9,1.8,9300,400,30,2.5,9300,1500,0.3,7.1]
        ,'HE0351+0240':[2.2,37,10600,77,1.1,1.5,10618,277,1.5,0.07,11580,320,3,0.35,10650,1344,-0.3,3]                                                    
        ,'HE0412-0803':[1.139,1.5,11448,50.0,0.1,0.3,11248,100.0,0.1,0.1,11448,1000.0,0,0,11448,1000.0,-0.001,0.5]
        ,'HE0429-0247':[0.9,8.5,12600,90,0.2,2.0,12500,220,1.7,0.07,12600,450,1.3,0.2,12600,1600,-0.5,4.0]
        ,'HE0433-1028':[0.9,4.0,10665.0,100.0,1.0,1.0,10465.0,300.0,1.0,1.0,10665,800.0,0.1,0.1,10665,1500.0,-0.1,1.0]
        ,'HE0853+0102':[0.1,0.9,15700,110.0,0.02,0.3,15670,320.0,0.1,0.1,15900,1640.0,0.0,0.0,17250,810.0,-0.001,0.1]
        ,'HE0934+0119':[0.6,1.9,15101,100.0,0.2,0.2,14901,400.0,0.3,0.1,15101,1000.0,0.1,0.1,15101,1000.0,0.001,0.7]
        ,'HE1011-0403':[0.1,0.6,17494,60.0,0.2,0.2,17244,200.0,0.3,0.1,17494,1500.0,0.1,0.1,17494,1500.0,0.001,0.7]
        ,'HE1017-0305':[0.1,1.5,14695,50,0.08,0.4,14595,200,0.7,0.09,14695,2000,0,0,0,0,-0.3,2.0]
        ,'HE1029-1831':[0.6,1.3,12078,50.0,0.2,0.2,11978,200.0,0.3,0.1,12078,1000.0,0.2,0.2,12078,1000.0,-0.001,0.2]
        ,'HE1107-0813':[0.04,0.3,17500,100,0.2,0.13,17400,480,0.9,0.13,17500,1000,0.9,0.08,16800,2000,-0.9,8]
        ,'HE1108-2813':[1.139,1.5,7200.0,50.0,1.0,3.0,7000.0,100.0,1.0,1.0,7200,1000.0,0,0,7200,1000.0,-0.001,2.0]
        ,'HE1126-0407':[1,13,18200,100,2.5,3,18000,300,10,1.3,18200,800,7,1.8,18200,1600,-0.9,15]
        ,'HE1237-0504':[1,10,2700.0,80.0,0.5,2,2700.0,200,2,0.5,2700.0,1500.0,0,0,2700.0,1000.0,-0.001,0.002]
        ,'HE1248-1356':[0.1,1.5,4395.0,50.0,1.0,3.0,4195.0,100.0,1.0,1.0,4395,1000.0,0,0,0,0.0,-0.001,2.0]
        ,'HE1330-1013':[0.15,0.6,6643,90,0.05,0.15,6543,200,0.12,0.04,6643,1500,0.3,0.04,6643,500,-0.02,0.3]
        ,'HE1353-1917':[0.07,0.7,10490.0,80.0,0.39,0.05,10306.0,490.0,0.1,0.02,8600.0,1500.0,0.14,0.02,12326.0,1500.0,-0.001,0.002]
        ,'HE1417-0909':[1,12.5,13200,50.0,3,3,13000,100.0,2,2,13200,1000.0,0,0,13200,1000.0,-0.001,0.1]
        ,'HE2128-0221':[0.1,1.5,15744,50.0,1.0,3.0,15544,100.0,1.0,1.0,15744,1000.0,0,0,15744,1000.0,-0.001,2.0]
        ,'HE2211-3903':[0.6,2.4,11914,50.0,0.1,0.1,11714,100.0,0.1,0.1,11914,200.0,0,0,11914,100.0,-0.001,0.2]
        ,'HE2222-0026':[0.08,0.4,17400.0,140.0,0.04,0.05,17150.0,300.0,0.4,0.1,18500,650.0,0.02,0.01,17460,1750.0,-0.001,0.002]
        ,'HE2233+0124':[0.1,1.2,16944.0,100.0,1.0,3.0,17044.0,300.0,1.0,1.0,16944,1200.0,1.0,1.0,16944,4000.0,-0.001,2.0]
        ,'HE2302-0857':[1,8,14058,200,0.1,2,14258,300,1,0.1,14058,1000,0,0,0,0,0.01,0.2]}

for obj in objs:
    (Hb1,Hb2,wing,core) = flux_dat(obj)
    box_size = np.shape(Hb1)[1]  
    y, x = np.mgrid[:box_size, :box_size]
    xy=(x,y)
    algorithm_script(obj,p_init[obj],z[obj],broad2[obj]) 


HE0021-1810


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in power


norm chi-sq err and flux_err is [0.29269932 2.92542417 2.89716092 1.60145449 1.59363369 0.05899778
 0.49384429 0.82315116 0.23149123 1.15413181 0.14460909 0.45748248
 0.58124135 0.13970368 0.14784039 0.16658644 0.03226565 0.04256475
 0.02548563 0.03718095 0.14604905 0.17322235 0.14576201 0.17538043]
HE0021-1819
norm chi-sq err and flux_err is [1.74171118e-01 3.86657689e-01 3.62189012e+00 2.51607056e-01
 2.12237249e+00 1.37044610e-03 8.35987985e-03 4.98016940e-02
 4.27543008e-02 6.02093439e-02 5.43192243e-03 7.72658503e-02
 7.66370500e-02 3.33726257e-02 2.40786805e-02 2.15165988e-02
 1.01380412e-02 1.00250426e-02 5.82326612e-03 7.30712769e-03
 2.62946582e-02 2.38130086e-02 2.43929341e-02 2.26906232e-02]
HE0040-1105
norm chi-sq err and flux_err is [1.90384325e-01 2.58193094e-01 1.14279331e+01 1.38960743e-01
 6.45857819e+00 2.48260261e-03 5.72038231e-03 2.95774497e-02
 8.86447327e-02 8.02840986e-02 3.18517842e-03 2.04962062e-02
 1.98952824e-02 1.20653851e-02 8.10854636e-03 6.58303744e-03


norm chi-sq err and flux_err is [7.46042006e-01 6.94297276e-01 9.63703394e+00 4.99272075e-02
 5.58925080e-01 6.60907128e-03 7.30203484e-03 4.00381957e-02
 9.80169156e-02 8.24346742e-02 1.63742389e-03 5.03438368e-03
 5.45338989e-03 5.41078421e-03 1.70554453e-03 1.83312581e-03
 5.01949478e-03 4.75849834e-03 2.19046957e-03 1.68496368e-03
 5.44012749e-03 4.99058852e-03 2.74348590e-03 2.49835251e-03]
HE1248-1356
norm chi-sq err and flux_err is [1.64849333e-01 6.79843548e-01 5.13364787e+00 4.04014508e-01
 2.68127429e+00 3.67092944e-03 1.43468449e-02 9.89879143e-02
 1.40972135e-01 2.07172022e-01 3.19608283e-02 5.37562646e-02
 6.77998089e-02 5.90796283e-02 2.30285491e-02 2.45363870e-02
 3.33426318e-02 2.94639988e-02 9.17186807e-03 8.68589555e-03
 4.02053156e-02 3.68620831e-02 2.37729472e-02 2.32326625e-02]
HE1330-1013
norm chi-sq err and flux_err is [0.14796903 0.13771706 0.40919986 0.08826039 0.27566052 0.01030549
 0.0062458  0.02164828 0.09163068 0.0844378  0.01262541 0.0823418
 0.08742989 0